In [1]:
import cv2
import os
import json

In [8]:
with open('../config.json') as file:
    config = dict(json.load(file))

In [9]:
# Load the pre-trained face detection cascade
face_cascade = cv2.CascadeClassifier(config['hc_path'])
eye_cascade = cv2.CascadeClassifier(config['eye_cascade'])

In [10]:
def extract_images_from_video(_path, _save_to):
    # Check if the save directory exists, create if it doesn't
    os.makedirs(_save_to, exist_ok=True)

    # Open the video file
    video = cv2.VideoCapture(_path)

    # Check if the video file is successfully opened
    if not video.isOpened():
        raise ("Error opening video file")

    count_of_images = 1
    while video.isOpened():
        # Read a frame from the video
        ret, frame = video.read()

        if ret:
            # Convert the frame to grayscale
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Perform face detection
            faces = face_cascade.detectMultiScale(
                gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for i, (x, y, w, h) in enumerate(faces):
                # Crop the detected face region
                face = gray_frame[y:y+h, x:x+w]

                eyes = eye_cascade.detectMultiScale(face)
                if (len(eyes) >= 2):
                    # Save the cropped face image
                    cv2.imwrite(
                        os.path.join(
                            _save_to,
                            f"frame_{count_of_images}_face_{i+1}.png"
                        ),
                        cv2.resize(
                            face, (
                                config['resize_w_h'],
                                config['resize_w_h']
                            )
                        )
                    )

            count_of_images += 1

        else:
            break

    # Release the video capture and close any open windows
    video.release()
    cv2.destroyAllWindows()

In [11]:
for name in config['members']:
    extract_images_from_video(config['raw'][name], config['save_to'][name])